The requirements are:
google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client, re, numpy

Please do:

pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread  numpy

to install them

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client numpy


In [2]:
from lib.youtube_handler import *
from lib.gsheet_manager import *
import time
import random

client_secrets_file = "my_client.json"  # Path to your client secrets file
youtube_manager = YouTubeManager(client_secrets_file)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=195848401684-rshqm3h42ijd18qvk9n619snq9nvo7tg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54824%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=b0r3ZU98xE9ZO8K9YSjoRFLZt7OkEM&access_type=offline


In [3]:
from lib.gsheet_manager import *
service_account_file = "sheet_client.json"
playlist_id = "PLP641Bf8QxbVdSd3qQGSePIx1HChpmfmQ"  # Replace with your actual playlist ID
debug = SheetDebugManager(service_account_file)
requests = SheetRequestManager(service_account_file)
queue = SheetQueueManager(service_account_file)

In [4]:
youtube_manager.remove_all_videos_from_playlist( playlist_id)

All videos have been removed from the playlist.


In [5]:
youtude_interplay = ["https://www.youtube.com/watch?v=owu16UvLqGo",
                     "https://www.youtube.com/watch?v=n8IiMSXqIAo",
                     "https://www.youtube.com/watch?v=lNgH9b4xq0c" ]

while True:
    auth_users    = SheetAuthorizedUsersManager(service_account_file).get_usernames()
    auth_singers  = SheetAuthorizedUsersManager(service_account_file).get_singers()
    users2singers = dict(list(zip(auth_users,auth_singers)))

    records = requests.read_records();
    if len(records)>0:
        for record in requests.read_records():
            #Validate user
            current_user = str(record['Usuario']).lower()
            if current_user in auth_users: #We validate proper user
                current_link = record['Link de Youtube']
                if youtube_manager.is_valid_youtube_link(current_link):
                    send_youtube =True                        
                    queue_record = [record["Marca temporal"],
                                    users2singers[current_user],
                                    record['Autor/Anime'],
                                    record['Titulo'],record['Link de Youtube']]
                    print(queue_record)
                    try:
                        queue.add_record(queue_record)
                    except:
                        debug.add_record(list(record.values()))
                        send_youtube =False                        
                    if send_youtube:
                        choice= random.randint(0,2)
                        youtube_manager.add_video_to_playlist(youtude_interplay[choice], playlist_id)
                        youtube_manager.add_video_to_playlist(current_link, playlist_id)                    
                else:
                    debug.add_record(list(record.values()))            
            else:
                debug.add_record(list(record.values()))
        requests.delete_all_records()
        
    time.sleep(10) # Sleep for 3 seconds


['4/02/2024 19:22:29', 'Miriam', 'Kimetsu no Yaiba', 'Zankyou Sanka', 'https://youtu.be/GT4WNcEbHGk?si=RG8JxA1xGjEfaI7N']
Added video to playlist: {'kind': 'youtube#playlistItem', 'etag': '6OE5SKtomYa4BmqS9Z2AsPrSw6A', 'id': 'UExQNjQxQmY4UXhiVmRTZDNxUUdTZVBJeDFIQ2hwbWZtUS41NkI0NEY2RDEwNTU3Q0M2', 'snippet': {'publishedAt': '2024-02-04T18:52:12Z', 'channelId': 'UCPUzd2HrXUPQGjt3VYvhwLw', 'title': 'kai interplay2 1', 'description': '', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/lNgH9b4xq0c/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/lNgH9b4xq0c/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/lNgH9b4xq0c/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/lNgH9b4xq0c/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/lNgH9b4xq0c/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'JoseH', 'playlistId': 'PLP6

KeyboardInterrupt: 

In [ ]:
youtube_manager.add_video_to_playlist(youtude_interplay[0], playlist_id)
                        

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet&alt=json returned "Playlist should use manual sorting to support position.". Details: "[{'message': 'Playlist should use manual sorting to support position.', 'domain': 'youtube.playlistItem', 'reason': 'manualSortRequired'}]">